In [1]:
from timebudget import timebudget
import numpy as np
import torch

from cplAE_snmCAT.models.torch_cplAE import Model_TE
from cplAE_snmCAT.utils.load_config import load_config
from cplAE_snmCAT.utils.proc import get_splits, select_dataset_v2
from cplAE_snmCAT.models.torch_cplAE import tonumpy, tensor_
from functools import partial

device='cpu'
astensor = partial(tensor_,device=device)

In [2]:
paths = load_config(verbose=False)
#Data selection===================
n_genes = 1000
data = select_dataset_v2(n_genes,
                        select_T='sorted_highvar_T_genes',
                        select_E='sorted_highvar_E_genes')
train_ind, val_ind = get_splits(data=data, fold=0, n_folds=10)
splits = {'train_ind': train_ind, 'val_ind': val_ind}

In [3]:
fname = 'Sep23_aT_1-0_aE_1-0_csTE_1-0_ad_1_ld_2_bs_500_ne_5000_out_1_ri_0_exit_weights.pt'
saved_weights_path = paths['package_dir'] / 'data/results/v2' / fname

In [4]:
store = torch.load(saved_weights_path, map_location=device)
model = Model_TE(**store['hparams'])
model.load_state_dict(store['model_state_dict'])
model.eval()

with timebudget('prediction'):
    with torch.no_grad():
        model.eval()
        zT, zE, XrT, XrE, _ = model((astensor(data['XT']), astensor(data['XE'])))

zT = tonumpy(zT)
zE = tonumpy(zE)
XrT = tonumpy(XrT)
XrE = tonumpy(XrE)

prediction took 103.69ms
